In [1]:
# import necessary libraries
import pandas as pd
import numpy as np
import glob
import re
pd.options.display.max_columns = None
pd.options.display.max_rows = 100

# Load Data
Import the 2019 TIMSS Data into six separate dataframes

In [20]:
school = 0
stud_ach = 0
student = 0
stud_teach = 0
math_teach = 0
sci_teach = 0
df_list = []
for file in glob.glob('T19_G8_SPSS Data/*.sav'):
    if re.search('bcg',file) != None:
        try:
            school = school.append(pd.read_spss(file), ignore_index=True, sort=False)
        except:
            school = pd.read_spss(file)
    elif re.search('bsa',file) != None:
        try:
            stud_ach = stud_ach.append(pd.read_spss(file), ignore_index=True, sort=False)
        except:
            stud_ach = pd.read_spss(file)
    elif re.search('bsg',file) != None:
        try:
            student = student.append(pd.read_spss(file), ignore_index=True, sort=False)
        except:
            student = pd.read_spss(file)
            student.name = 'student'
            df_list.append(student)
    elif re.search('bst',file) != None:
        try:
            stud_teach = stud_teach.append(pd.read_spss(file), ignore_index=True, sort=False)
        except:
            stud_teach = pd.read_spss(file)
            stud_teach.name = 'stud_teach'
            df_list.append(stud_teach)
    elif re.search('btm',file) != None:
        try:
            math_teach = math_teach.append(pd.read_spss(file), ignore_index=True, sort=False)
        except:
            math_teach = pd.read_spss(file)
            math_teach.name = 'math_teach'
            df_list.append(math_teach)
    elif re.search('bts',file) != None:
        try:
            sci_teach = sci_teach.append(pd.read_spss(file), ignore_index=True, sort=False)
        except:
            sci_teach = pd.read_spss(file)
            sci_teach.name = 'sci_teach'
            df_list.append(sci_teach)

In [21]:
school.shape

(9925, 98)

In [22]:
stud_ach.shape

(295453, 2073)

In [23]:
student.shape

(295453, 455)

In [24]:
stud_teach.shape

(775679, 115)

In [25]:
math_teach.shape

(14571, 158)

In [26]:
sci_teach.shape

(22648, 169)

In [31]:
school.name = 'school'
stud_ach.name = 'stud_ach'
student.name = 'student'
stud_teach.name = 'stud_teach'
math_teach.name = 'math_teach'
sci_teach.name = 'sci_teach'

In [29]:
school.to_csv('data/{}.csv'.format(school.name), index = False)

In [30]:
stud_ach.to_csv('data/{}.csv'.format(stud_ach.name), index = False)

In [32]:
student.to_csv('data/{}.csv'.format(student.name), index = False)
stud_teach.to_csv('data/{}.csv'.format(stud_teach.name), index = False)
math_teach.to_csv('data/{}.csv'.format(math_teach.name), index = False)
sci_teach.to_csv('data/{}.csv'.format(sci_teach.name), index = False)

In [ ]:
school = pd.read_csv('data/school.csv')

In [ ]:
stud_ach = pd.read_csv('data/stud_ach.csv')

In [ ]:
student = pd.read_csv('data/student.csv')

In [73]:
stud_teach = pd.read_csv('data/stud_teach.csv')

In [ ]:
math_teach = pd.read_csv('data/math_teach.csv')

In [ ]:
sci_teach = pd.read_csv('data/sci_teach.csv')

In [74]:
stud_teach.iloc[100:109,:]

,IDCNTRY,IDBOOK,IDSCHOOL,IDCLASS,IDSTUD,IDTEALIN,IDTEACH,IDLINK,IDPOP,IDGRADER,IDGRADE,IDSUBJ,ITCOURSE,MATSUBJ,SCISUBJ,NMTEACH,NSTEACH,NTEACH,MATWGT,SCIWGT,TCHWGT,JKREP,JKZONE,BSMMAT01,BSMMAT02,BSMMAT03,BSMMAT04,BSMMAT05,BSSSCI01,BSSSCI02,BSSSCI03,BSSSCI04,BSSSCI05,BSMALG01,BSMALG02,BSMALG03,BSMALG04,BSMALG05,BSMAPP01,BSMAPP02,BSMAPP03,BSMAPP04,BSMAPP05,BSMDAT01,BSMDAT02,BSMDAT03,BSMDAT04,BSMDAT05,BSMGEO01,BSMGEO02,BSMGEO03,BSMGEO04,BSMGEO05,BSMKNO01,BSMKNO02,BSMKNO03,BSMKNO04,BSMKNO05,BSMNUM01,BSMNUM02,BSMNUM03,BSMNUM04,BSMNUM05,BSMREA01,BSMREA02,BSMREA03,BSMREA04,BSMREA05,BSSAPP01,BSSAPP02,BSSAPP03,BSSAPP04,BSSAPP05,BSSBIO01,BSSBIO02,BSSBIO03,BSSBIO04,BSSBIO05,BSSCHE01,BSSCHE02,BSSCHE03,BSSCHE04,BSSCHE05,BSSEAR01,BSSEAR02,BSSEAR03,BSSEAR04,BSSEAR05,BSSKNO01,BSSKNO02,BSSKNO03,BSSKNO04,BSSKNO05,BSSPHY01,BSSPHY02,BSSPHY03,BSSPHY04,BSSPHY05,BSSREA01,BSSREA02,BSSREA03,BSSREA04,BSSREA05,BSMIBM01,BSMIBM02,BSMIBM03,BSMIBM04,BSMIBM05,BSSIBM01,BSSIBM02,BSSIBM03,BSSIBM04,BSSIBM05,VERSION,SCOPE
100,7842.0,Booklet 11,5004.0,500402.0,50040223.0,50040202.0,500402.0,2.0,Pop 2,Upper Grade,Grade 8,Science,Integrated science,0.0,1.0,1.0,1.0,2.0,0.00,3.48,1.74,0.0,69.0,389.71679,263.61548,388.60165,365.63210,355.35784,299.28696,138.60930,181.12159,184.21837,215.53681,356.63090,356.66227,380.31519,403.31116,418.27206,352.84221,359.77229,262.59729,367.54897,381.75764,324.03407,329.64790,320.47479,316.66042,414.62597,269.23235,248.94190,244.97052,276.04388,312.73906,343.53740,355.68452,319.22921,366.05853,380.91427,329.97110,306.79900,361.34040,355.49213,382.79590,343.77681,368.97853,281.44643,386.32186,396.63025,99.54778,141.79248,233.57407,247.86414,237.99465,223.98821,178.31289,224.33362,241.27830,135.60613,237.10378,195.56012,209.76439,272.80608,166.79001,292.01292,233.84250,266.68947,282.12397,204.38574,114.36273,153.34909,305.32611,270.70890,285.06008,280.03879,212.45848,262.34694,294.02052,163.86998,113.99643,183.13423,260.22236,329.60844,249.23554,Below 400,Below 400,Below 400,Below 400,Below 400,Below 400,Below 400,Below 400,Below 400,Below 400,4.0,Public Use File (PUF)
101,7842.0,Booklet 11,5004.0,500402.0,50040223.0,50040101.0,500401.0,1.0,Pop 2,Upper Grade,Grade 8,Mathematics,Mathematics,1.0,0.0,1.0,1.0,2.0,3.48,0.00,1.74,0.0,69.0,389.71679,263.61548,388.60165,365.63210,355.35784,299.28696,138.60930,181.12159,184.21837,215.53681,356.63090,356.66227,380.31519,403.31116,418.27206,352.84221,359.77229,262.59729,367.54897,381.75764,324.03407,329.64790,320.47479,316.66042,414.62597,269.23235,248.94190,244.97052,276.04388,312.73906,343.53740,355.68452,319.22921,366.05853,380.91427,329.97110,306.79900,361.34040,355.49213,382.79590,343.77681,368.97853,281.44643,386.32186,396.63025,99.54778,141.79248,233.57407,247.86414,237.99465,223.98821,178.31289,224.33362,241.27830,135.60613,237.10378,195.56012,209.76439,272.80608,166.79001,292.01292,233.84250,266.68947,282.12397,204.38574,114.36273,153.34909,305.32611,270.70890,285.06008,280.03879,212.45848,262.34694,294.02052,163.86998,113.99643,183.13423,260.22236,329.60844,249.23554,Below 400,Below 400,Below 400,Below 400,Below 400,Below 400,Below 400,Below 400,Below 400,Below 400,4.0,Public Use File (PUF)
102,7842.0,Booklet 12,5004.0,500402.0,50040224.0,50040101.0,500401.0,1.0,Pop 2,Upper Grade,Grade 8,Mathematics,Mathematics,1.0,0.0,1.0,1.0,2.0,3.48,0.00,1.74,0.0,69.0,231.91803,234.32788,339.22812,323.54287,328.49135,92.88993,244.33778,274.92431,169.44479,209.82736,256.17104,401.42878,220.33385,275.96361,302.91902,379.95388,292.55019,254.31106,372.03184,245.12685,260.57217,258.62386,125.53359,211.21960,116.39645,296.31585,294.91333,204.68430,237.06724,223.90999,343.96544,331.92785,272.93164,388.40242,268.23255,318.67986,349.31310,224.24435,293.79661,268.72664,325.06698,238.70708,213.38380,343.05374,235.95135,136.84894,286.59248,222.43742,264.73853,209.85316,297.12408,54.19767,269.38016,180.50992,151.03476,270.93014,64.15481,197.09609,165.77387,106.04646,314.10447,66.33654,181.95234,165.06569,149.33758,

In [75]:
stud_teach[stud_teach.IDTEACH.astype(int)*100+stud_teach.IDLINK.astype(int) != stud_teach.IDTEALIN.astype(int)]

,IDCNTRY,IDBOOK,IDSCHOOL,IDCLASS,IDSTUD,IDTEALIN,IDTEACH,IDLINK,IDPOP,IDGRADER,IDGRADE,IDSUBJ,ITCOURSE,MATSUBJ,SCISUBJ,NMTEACH,NSTEACH,NTEACH,MATWGT,SCIWGT,TCHWGT,JKREP,JKZONE,BSMMAT01,BSMMAT02,BSMMAT03,BSMMAT04,BSMMAT05,BSSSCI01,BSSSCI02,BSSSCI03,BSSSCI04,BSSSCI05,BSMALG01,BSMALG02,BSMALG03,BSMALG04,BSMALG05,BSMAPP01,BSMAPP02,BSMAPP03,BSMAPP04,BSMAPP05,BSMDAT01,BSMDAT02,BSMDAT03,BSMDAT04,BSMDAT05,BSMGEO01,BSMGEO02,BSMGEO03,BSMGEO04,BSMGEO05,BSMKNO01,BSMKNO02,BSMKNO03,BSMKNO04,BSMKNO05,BSMNUM01,BSMNUM02,BSMNUM03,BSMNUM04,BSMNUM05,BSMREA01,BSMREA02,BSMREA03,BSMREA04,BSMREA05,BSSAPP01,BSSAPP02,BSSAPP03,BSSAPP04,BSSAPP05,BSSBIO01,BSSBIO02,BSSBIO03,BSSBIO04,BSSBIO05,BSSCHE01,BSSCHE02,BSSCHE03,BSSCHE04,BSSCHE05,BSSEAR01,BSSEAR02,BSSEAR03,BSSEAR04,BSSEAR05,BSSKNO01,BSSKNO02,BSSKNO03,BSSKNO04,BSSKNO05,BSSPHY01,BSSPHY02,BSSPHY03,BSSPHY04,BSSPHY05,BSSREA01,BSSREA02,BSSREA03,BSSREA04,BSSREA05,BSMIBM01,BSMIBM02,BSMIBM03,BSMIBM04,BSMIBM05,BSSIBM01,BSSIBM02,BSSIBM03,BSSIBM04,BSSIBM05,VERSION,SCOPE


In [76]:
stud_teach.drop('IDTEALIN', axis = 1, inplace=True)

In [77]:
stud_teach.head(1)

,IDCNTRY,IDBOOK,IDSCHOOL,IDCLASS,IDSTUD,IDTEACH,IDLINK,IDPOP,IDGRADER,IDGRADE,IDSUBJ,ITCOURSE,MATSUBJ,SCISUBJ,NMTEACH,NSTEACH,NTEACH,MATWGT,SCIWGT,TCHWGT,JKREP,JKZONE,BSMMAT01,BSMMAT02,BSMMAT03,BSMMAT04,BSMMAT05,BSSSCI01,BSSSCI02,BSSSCI03,BSSSCI04,BSSSCI05,BSMALG01,BSMALG02,BSMALG03,BSMALG04,BSMALG05,BSMAPP01,BSMAPP02,BSMAPP03,BSMAPP04,BSMAPP05,BSMDAT01,BSMDAT02,BSMDAT03,BSMDAT04,BSMDAT05,BSMGEO01,BSMGEO02,BSMGEO03,BSMGEO04,BSMGEO05,BSMKNO01,BSMKNO02,BSMKNO03,BSMKNO04,BSMKNO05,BSMNUM01,BSMNUM02,BSMNUM03,BSMNUM04,BSMNUM05,BSMREA01,BSMREA02,BSMREA03,BSMREA04,BSMREA05,BSSAPP01,BSSAPP02,BSSAPP03,BSSAPP04,BSSAPP05,BSSBIO01,BSSBIO02,BSSBIO03,BSSBIO04,BSSBIO05,BSSCHE01,BSSCHE02,BSSCHE03,BSSCHE04,BSSCHE05,BSSEAR01,BSSEAR02,BSSEAR03,BSSEAR04,BSSEAR05,BSSKNO01,BSSKNO02,BSSKNO03,BSSKNO04,BSSKNO05,BSSPHY01,BSSPHY02,BSSPHY03,BSSPHY04,BSSPHY05,BSSREA01,BSSREA02,BSSREA03,BSSREA04,BSSREA05,BSMIBM01,BSMIBM02,BSMIBM03,BSMIBM04,BSMIBM05,BSSIBM01,BSSIBM02,BSSIBM03,BSSIBM04,BSSIBM05,VERSION,SCOPE
0,7842.0,Booklet 11,5001.0,500102.0,50010201.0,500101.0,1.0,Pop 2,Upper Grade,Grade 8,Mathematics,Mathematics,1.0,0.0,1.0,1.0,2.0,1.8,0.0,0.9,1.0,31.0,536.92562,536.18211,553.22063,569.69968,556.72617,477.42915,459.80829,462.51871,525.67103,475.07255,516.13192,493.67505,543.56644,547.2563,484.58233,469.19131,518.54136,478.872,514.33301,533.24322,472.37213,491.41828,499.53291,483.62696,481.43969,461.46762,514.12465,523.61825,546.79913,494.8351,500.45313,546.97269,507.21281,529.93599,553.51881,541.61237,513.85035,565.12265,539.91102,515.3146,501.03317,523.50152,525.08347,503.83817,532.55965,492.16212,466.83672,428.39092,443.0282,465.7187,417.9647,385.59263,472.90277,366.72065,379.40684,474.66426,436.89059,506.8122,358.13981,440.24602,420.042,367.86784,476.61587,364.86689,431.34778,496.46487,439.73289,476.37783,460.58785,484.737,474.31902,418.86673,501.97838,396.69597,461.53759,480.64801,452.8109,439.85092,463.09426,485.17843,At or above 475 but below 550,At or above 475 but below 550,At or above 550 but below 625,At or above 550 but below 625,At or above 550 but below 625,At or above 475 but below 550,At or above 400 but below 475,At or above 400 but below 475,At or above 475 but below 550,At or above 475 but below 550,4.0,Public Use File (PUF)


In [78]:
stud_teach.IDGRADER.unique()

array(['Upper Grade'], dtype=object)

In [79]:
stud_teach.IDGRADE.unique()

array(['Grade 8', 'Grade 9'], dtype=object)

In [80]:
stud_teach.IDSUBJ.unique()

array(['Mathematics', 'Science'], dtype=object)

In [81]:
stud_teach.ITCOURSE.unique()

array(['Mathematics', 'Integrated science', 'Physics', 'Biology',
       'Chemistry', 'Earth science', 'Biology/Chemistry',
       'Physics/Chemistry', 'Biology/Earth science'], dtype=object)

In [82]:
stud_teach.ITCOURSE.value_counts()

Mathematics              303798
Integrated science       250575
Biology                   48396
Physics                   48195
Chemistry                 47960
Earth science             33592
Physics/Chemistry         21813
Biology/Earth science     21308
Biology/Chemistry            42
Name: ITCOURSE, dtype: int64

In [83]:
stud_teach.NMTEACH.unique()

array([1., 2., 3., 5., 6., 4., 0.])

In [84]:
stud_teach.NSTEACH.unique()

array([ 1.,  3.,  2., nan,  4.,  5.,  6.,  0.,  7.,  8.])

In [85]:
stud_teach.NTEACH.unique()

array([ 2.,  4.,  3.,  6.,  8.,  9., 10.,  7.,  5.,  1.])

In [86]:
stud_teach.loc[:,sub_col].mean(axis=1)

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
          ..
775674   NaN
775675   NaN
775676   NaN
775677   NaN
775678   NaN
Length: 775679, dtype: float64

In [87]:
scores = ['MMAT','SSCI','MALG','MAPP','MDAT','MGEO','MKNO','MNUM','MREA','SAPP','SBIO','SCHE','SEAR','SKNO','SPHY','SREA']
for item in scores:
    sub_col = [col for col in stud_teach.columns if item in col]
    print(sub_col)
    stud_teach[item+'_avg'] = stud_teach.loc[:,sub_col].mean(axis=1)
    stud_teach[item+'_sd'] = stud_teach.loc[:,sub_col].std(axis=1)
    stud_teach.drop(sub_col, axis = 1, inplace=True)

['BSMMAT01', 'BSMMAT02', 'BSMMAT03', 'BSMMAT04', 'BSMMAT05']
['BSSSCI01', 'BSSSCI02', 'BSSSCI03', 'BSSSCI04', 'BSSSCI05']
['BSMALG01', 'BSMALG02', 'BSMALG03', 'BSMALG04', 'BSMALG05']
['BSMAPP01', 'BSMAPP02', 'BSMAPP03', 'BSMAPP04', 'BSMAPP05']
['BSMDAT01', 'BSMDAT02', 'BSMDAT03', 'BSMDAT04', 'BSMDAT05']
['BSMGEO01', 'BSMGEO02', 'BSMGEO03', 'BSMGEO04', 'BSMGEO05']
['BSMKNO01', 'BSMKNO02', 'BSMKNO03', 'BSMKNO04', 'BSMKNO05']
['BSMNUM01', 'BSMNUM02', 'BSMNUM03', 'BSMNUM04', 'BSMNUM05']
['BSMREA01', 'BSMREA02', 'BSMREA03', 'BSMREA04', 'BSMREA05']
['BSSAPP01', 'BSSAPP02', 'BSSAPP03', 'BSSAPP04', 'BSSAPP05']
['BSSBIO01', 'BSSBIO02', 'BSSBIO03', 'BSSBIO04', 'BSSBIO05']
['BSSCHE01', 'BSSCHE02', 'BSSCHE03', 'BSSCHE04', 'BSSCHE05']
['BSSEAR01', 'BSSEAR02', 'BSSEAR03', 'BSSEAR04', 'BSSEAR05']
['BSSKNO01', 'BSSKNO02', 'BSSKNO03', 'BSSKNO04', 'BSSKNO05']
['BSSPHY01', 'BSSPHY02', 'BSSPHY03', 'BSSPHY04', 'BSSPHY05']
['BSSREA01', 'BSSREA02', 'BSSREA03', 'BSSREA04', 'BSSREA05']


In [101]:
col_set = ['IDGRADER','MATSUBJ','NTEACH','SCISUBJ','MATWGT','SCIWGT','TCHWGT','JKREP','JKZONE']
for col in stud_teach.columns:
    if 'IBM' in col:
        col_set.append(col)
print(col_set)
stud_teach.drop(col_set, axis=1, inplace=True)

['IDGRADER', 'MATSUBJ', 'NTEACH', 'SCISUBJ', 'MATWGT', 'SCIWGT', 'TCHWGT', 'JKREP', 'JKZONE', 'BSMIBM01', 'BSMIBM02', 'BSMIBM03', 'BSMIBM04', 'BSMIBM05', 'BSSIBM01', 'BSSIBM02', 'BSSIBM03', 'BSSIBM04', 'BSSIBM05']


In [103]:
stud_teach.shape

(775679, 47)

In [92]:
stud_ach.shape

(295453, 2073)

In [91]:
stud_ach.head()

IDCNTRY      IDBOOK  IDSCHOOL   IDCLASS      IDSTUD MP52024 MP52058A  \
0   7842.0  Booklet 11    5001.0  500102.0  50010201.0     NaN      NaN   
1   7842.0  Booklet 12    5001.0  500102.0  50010202.0     NaN      NaN   
2   7842.0  Booklet 13    5001.0  500102.0  50010203.0     NaN      NaN   
3   7842.0  Booklet 14    5001.0  500102.0  50010204.0     NaN      NaN   
4   7842.0  Booklet 02    5001.0  500102.0  50010207.0     NaN      NaN   

  MP52058B MP52125 MP52229 MP52063 MP52072 MP52146A MP52146B MP52092 MP52046  \
0      NaN     NaN     NaN     NaN     NaN      NaN      NaN     NaN     NaN   
1      NaN     NaN     NaN     NaN     NaN      NaN      NaN     NaN     NaN   
2      NaN     NaN     NaN     NaN     NaN      NaN      NaN     NaN     NaN   
3      NaN     NaN     NaN     NaN     NaN      NaN      NaN     NaN     NaN   
4      NaN     NaN     NaN     NaN     NaN      NaN      NaN     NaN     NaN   

  MP52083 MP52082 MP52161 MP52418A MP52418B MP72007A MP72007B MP72007C  \
0     NaN     NaN     NaN      NaN      NaN      NaN      NaN      NaN   
1     NaN     NaN     NaN      NaN      NaN      NaN      NaN      NaN   
2     NaN     NaN     NaN      NaN      NaN      NaN      NaN      NaN   
3     NaN     NaN     NaN      NaN      NaN      NaN      NaN      NaN   
4     NaN     NaN     NaN      NaN      NaN      NaN      NaN      NaN   

  MP72007D MP72007E MP72007 MP72025 MP72017 MP72190 MP72068 MP72076 MP72056  \
0      NaN      NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
1      NaN      NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
2      NaN      NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
3      NaN      NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
4      NaN      NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   

  MP72098 MP72103 MP72121 MP72180 MP72198A MP72198B MP72198 MP72227 MP72170  \
0     NaN     NaN     NaN     NaN      NaN      NaN     NaN     NaN     NaN   
1     NaN     NaN     NaN     NaN      NaN      NaN     NaN     NaN     NaN   
2     NaN     NaN     NaN     NaN      NaN      NaN     NaN     NaN     NaN   
3     NaN     NaN     NaN     NaN      NaN      NaN     NaN     NaN     NaN   
4     NaN     NaN     NaN     NaN      NaN      NaN     NaN     NaN     NaN   

  MP72209 MP62005 MP62139 MP62164 MP62142 MP62084 MP62351 MP62223 MP62027  \
0     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
1     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
2     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
3     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
4     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   

  MP62174 MP62244 MP62261 MP62300 MP62254 MP62132A MP62132B MP72178 MP72234  \
0     NaN     NaN     NaN     NaN     NaN      NaN      NaN     NaN     NaN   
1     NaN     NaN     NaN     NaN     NaN      NaN      NaN     NaN     NaN   
2     NaN     NaN     NaN     NaN     NaN      NaN      NaN     NaN     NaN   
3     NaN     NaN     NaN     NaN     NaN      NaN      NaN     NaN     NaN   
4     NaN     NaN     NaN     NaN     NaN      NaN      NaN     NaN     NaN   

  MP72020 MP72027 MP72052A MP72052B MP72052 MP72067 MP72083A MP72083B  \
0     NaN     NaN      NaN      NaN     NaN     NaN      NaN      NaN   
1     NaN     NaN      NaN      NaN     NaN     NaN      NaN      NaN   
2     NaN     NaN      NaN      NaN     NaN     NaN      NaN      NaN   
3     NaN     NaN      NaN      NaN     NaN     NaN      NaN      NaN   
4     NaN     NaN      NaN      NaN     NaN     NaN      NaN      NaN   

  MP72108A MP72108B MP72181 MP72126 MP72164A MP72164B MP72164C MP72164D  \
0      NaN      NaN     NaN     NaN      NaN      NaN      NaN      NaN   
1      NaN      NaN     NaN     NaN      NaN      NaN      NaN      NaN   
2      NaN      NaN     NaN     NaN      NaN      NaN      NaN      NaN   

In [94]:
scores = ['MMAT','SSCI','MALG','MAPP','MDAT','MGEO','MKNO','MNUM','MREA','SAPP','SBIO','SCHE','SEAR','SKNO','SPHY','SREA']
for item in scores:
    sub_col = [col for col in stud_ach.columns if item in col]
    print(sub_col)
    stud_ach[item.lower()+'_avg'] = stud_ach.loc[:,sub_col].mean(axis=1)
    stud_ach[item.lower()+'_sd'] = stud_ach.loc[:,sub_col].std(axis=1)
    stud_ach.drop(sub_col, axis = 1, inplace=True)

sub_col = [col for col in stud_ach.columns if ('ME' in col and '_F' in col)]
print(sub_col)
stud_ach['math_freq_avg'] = stud_ach.loc[:,sub_col].mean(axis=1)
stud_ach['math_freq_sd'] = stud_ach.loc[:,sub_col].std(axis=1)
stud_ach.drop(sub_col, axis = 1, inplace=True)

sub_col = [col for col in stud_ach.columns if ('SE' in col and '_F' in col)]
print(sub_col)
stud_ach['sci_freq_avg'] = stud_ach.loc[:,sub_col].mean(axis=1)
stud_ach['sci_freq_sd'] = stud_ach.loc[:,sub_col].std(axis=1)
stud_ach.drop(sub_col, axis = 1, inplace=True)

sub_col = [col for col in stud_ach.columns if ('ME' in col and '_S' in col)]
print(sub_col)
stud_ach['math_time_avg'] = stud_ach.loc[:,sub_col].mean(axis=1)
stud_ach['math_time_sd'] = stud_ach.loc[:,sub_col].std(axis=1)
stud_ach.drop(sub_col, axis = 1, inplace=True)

sub_col = [col for col in stud_ach.columns if ('SE' in col and '_S' in col)]
print(sub_col)
stud_ach['sci_time_avg'] = stud_ach.loc[:,sub_col].mean(axis=1)
stud_ach['sci_time_sd'] = stud_ach.loc[:,sub_col].std(axis=1)
stud_ach.drop(sub_col, axis = 1, inplace=True)

['BSMMAT01', 'BSMMAT02', 'BSMMAT03', 'BSMMAT04', 'BSMMAT05']
['BSSSCI01', 'BSSSCI02', 'BSSSCI03', 'BSSSCI04', 'BSSSCI05']
['BSMALG01', 'BSMALG02', 'BSMALG03', 'BSMALG04', 'BSMALG05']
['BSMAPP01', 'BSMAPP02', 'BSMAPP03', 'BSMAPP04', 'BSMAPP05']
['BSMDAT01', 'BSMDAT02', 'BSMDAT03', 'BSMDAT04', 'BSMDAT05']
['BSMGEO01', 'BSMGEO02', 'BSMGEO03', 'BSMGEO04', 'BSMGEO05']
['BSMKNO01', 'BSMKNO02', 'BSMKNO03', 'BSMKNO04', 'BSMKNO05']
['BSMNUM01', 'BSMNUM02', 'BSMNUM03', 'BSMNUM04', 'BSMNUM05']
['BSMREA01', 'BSMREA02', 'BSMREA03', 'BSMREA04', 'BSMREA05']
['BSSAPP01', 'BSSAPP02', 'BSSAPP03', 'BSSAPP04', 'BSSAPP05']
['BSSBIO01', 'BSSBIO02', 'BSSBIO03', 'BSSBIO04', 'BSSBIO05']
['BSSCHE01', 'BSSCHE02', 'BSSCHE03', 'BSSCHE04', 'BSSCHE05']
['BSSEAR01', 'BSSEAR02', 'BSSEAR03', 'BSSEAR04', 'BSSEAR05']
['BSSKNO01', 'BSSKNO02', 'BSSKNO03', 'BSSKNO04', 'BSSKNO05']
['BSSPHY01', 'BSSPHY02', 'BSSPHY03', 'BSSPHY04', 'BSSPHY05']
['BSSREA01', 'BSSREA02', 'BSSREA03', 'BSSREA04', 'BSSREA05']
['ME52024_F', 'ME52058_F

In [95]:
stud_ach.shape

(295453, 1239)

In [105]:
stud_ach.to_csv('data/stud_ach.csv', index = False)
stud_teach.to_csv('data/stud_teach.csv', index = False)